In [ ]:
!pip install streamlit pyngrok --quiet

In [ ]:
!ngrok config add-authtoken WE2SPUJNXCPUEAVTICCJHKXQONGVHJ3Q


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!ngrok config add-authtoken 2vwIQrwjPio3L9KC2DeYe8vtlW4_5tzPNHZ6iG3BfrhojeWbK


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
pip install pytest requests-mock

In [ ]:
!pip install plotly

In [ ]:
cities = [
    "Edinburgh", "Glasgow", "Dundee", "Aberdeen", "Inverness",
    "Livingston", "Stirling", "Dunfermline", "Falkirk", "Perth"
]


In [ ]:
import requests
import pandas as pd
import plotly.express as px
import time

WAQI_TOKEN = "09591e540fc00070ffdf88e92ad50713a635a11a"

def fetch_city_data(city):
    try:
        url = f"https://api.waqi.info/feed/{city}/?token={WAQI_TOKEN}"
        response = requests.get(url)
        data = response.json()
        if data["status"] == "ok":
            return {
                "City": data["data"]["city"]["name"],
                "AQI": data["data"]["aqi"],
                "PM2.5": data["data"]["iaqi"].get("pm25", {}).get("v", None),
                "PM10": data["data"]["iaqi"].get("pm10", {}).get("v", None),
                "CO": data["data"]["iaqi"].get("co", {}).get("v", None),
                "NO2": data["data"]["iaqi"].get("no2", {}).get("v", None),
                "Dominant": data["data"].get("dominentpol", "N/A")
            }
    except Exception as e:
        print(f"Error fetching data for {city}: {e}")
    return None

# Fetch for all cities
aqi_data = []
for city in cities:
    result = fetch_city_data(city)
    if result:
        aqi_data.append(result)
    time.sleep(1)  # avoid rate limiting

df_aqi = pd.DataFrame(aqi_data)
df_aqi


,City,AQI,PM2.5,PM10,CO,NO2,Dominant
0,"Edinburgh St Leonards, United Kingdom",30,18,9,1.4,2.0,o3
1,"Glasgow Kerbside, United Kingdom",22,22,10,NaN,11.6,pm25
2,"Dundee Meadowside, United Kingdom",11,11,4,NaN,17.4,pm25
3,"Aberdeen Union Street Roadside, United Kingdom",6,6,3,NaN,3.5,pm25
4,"Inverness, United Kingdom",14,14,5,NaN,1.9,pm25
5,"West Lothian Broxburn, United Kingdom",31,31,17,NaN,2.3,pm25
6,"Stirling Craig's Roundabout, United Kingdom",13,13,7,NaN,5.9,pm25
7,"Fife Dunfermline, United Kingdom",19,19,6,NaN,1.7,pm25
8,"Falkirk Hope St, United Kingdom",22,22,7,NaN,4.4,pm25
9,"Perth Atholl Street, United Kingdom",29,29,10,NaN,8.8,pm25


In [63]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Example DataFrame from earlier (replace with your actual df_aqi if available)
df_aqi = pd.DataFrame({
    "City": ["Edinburgh", "Glasgow", "Dundee", "Aberdeen", "Inverness", "Livingston", "Stirling", "Dunfermline", "Falkirk", "Perth"],
    "AQI": [95, 110, 88, 120, 105, 92, 101, 98, 86, 90],
    "PM2.5": [60, 80, 58, 95, 76, 63, 70, 66, 55, 59],
    "PM10": [45, 70, 50, 85, 65, 48, 55, 50, 45, 47],
    "CO": [0.6, 0.8, 0.5, 1.0, 0.7, 0.6, 0.65, 0.62, 0.55, 0.58],
    "NO2": [25, 30, 22, 35, 28, 24, 26, 25, 21, 23],
    "Dominant": ["pm25", "pm25", "pm25", "pm10", "pm25", "pm25", "pm10", "pm25", "pm25", "pm25"]
})

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("AQI Comparison", "PM2.5 vs PM10", "Pollutant Levels"),
    specs=[[{"type": "bar"}, {"type": "scatter"}],
           [{"colspan": 2}, None]]
)

# Subplot 1: AQI Bar Chart
fig.add_trace(go.Bar(
    x=df_aqi["City"],
    y=df_aqi["AQI"],
    marker_color="indianred",
    name="AQI",
    text=df_aqi["AQI"],
    textposition="outside"
), row=1, col=1)

# Subplot 2: PM2.5 vs PM10 Scatter Bubble
fig.add_trace(go.Scatter(
    x=df_aqi["PM2.5"],
    y=df_aqi["PM10"],
    mode='markers+text',
    text=df_aqi["City"],
    marker=dict(size=df_aqi["AQI"], sizemode='area', sizeref=2.*max(df_aqi["AQI"])/(40.**2), sizemin=4),
    name="PM2.5 vs PM10"
), row=1, col=2)

# Subplot 3: Pollutants Stacked/Grouped Bar
df_melt = df_aqi[["City", "PM2.5", "PM10", "CO", "NO2"]].melt(id_vars="City", var_name="Pollutant", value_name="Value")
for pollutant in df_melt["Pollutant"].unique():
    sub_df = df_melt[df_melt["Pollutant"] == pollutant]
    fig.add_trace(go.Bar(
        x=sub_df["City"],
        y=sub_df["Value"],
        name=pollutant
    ), row=2, col=1)

# Update layout
fig.update_layout(
    title="🌍 Air Quality Dashboard - Top 10 Cities Around Edinburgh",
    height=600,
    showlegend=True,
    barmode='group'
)

fig.show()

